# **Importación de Librerías**

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import mlflow
import optuna
import os
import pprint
import time
import torch
import torch.nn as nn
import torch.optim as optim

from constants import *

from imblearn.over_sampling import SMOTE

from optuna.visualization import plot_pareto_front
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState

from sklearn.utils.class_weight import compute_class_weight

from utils.data_preprocessor import DataPreprocessor
from utils.load_data import *
from utils.train_predict import train_step, validate_step, validate_step_with_metrics, predict
from utils.metrics import *
from utils.optuna_utils import *
from utils.mlflow_logger import MLflowLogger

from models.instantiate import instantiate_model
from models.lstm_v1 import *
from models.lstm_v2 import *
from models.gru import *
from models.dense import *
from models.fcn import *

from tqdm.notebook import tqdm

np.random.seed(13)
torch.manual_seed(13)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
with open("config.json", "r") as f:
    config = json.load(f)

pprint.pp(config)

# PARTITIONS = config['partitions']
GROUP = "Grupo" + config['group']
SIMULATION = "Simulacion" + config['simulation']
REQ_PERIODS = config['required_periods']
METRICS = config['metrics']
DIRECTIONS = config['directions']
BETA = config['beta']
MODEL_ARCH = config['model_arch']
LOG_TO_MLFLOW = (config['log_to_mlflow'] == "True")

TEMP_FEATS = [f'y(t-{i})' for i in range(REQ_PERIODS, 0, -1)]
STAT_FEATS = ['inicio_prog']
FEATS = STAT_FEATS + TEMP_FEATS

In [ ]:
print(f"Log to MLflow: {LOG_TO_MLFLOW}")
mlflow_logger = MLflowLogger(
    LOG_TO_MLFLOW,
    TRACKING_SERVER_URI,
    EXPERIMENT_NAME,
    EXPERIMENT_TAGS
)

In [ ]:
GROUP_DIR = os.path.join(DATA_DIR, GROUP)
GROUP_PARAMS_FILE = os.path.join(GROUP_DIR, f"params_{GROUP}.json")
if os.path.exists(GROUP_PARAMS_FILE):
    with open(GROUP_PARAMS_FILE, 'r') as f:
        group_params = json.load(f)
        mlflow_logger.log_params(group_params)
else:
    print(f"Group params file not found: {GROUP_PARAMS_FILE}")

In [ ]:
stata_filepath = os.path.join(GROUP_DIR, SIMULATION)
if os.path.exists(stata_filepath):
    df = pd.read_stata(stata_filepath)
else:
    print(f"File {stata_filepath} not found.")

In [ ]:
mlflow_logger.log_params({
    "group": GROUP,
    "simulation": SIMULATION,
    "filepath": stata_filepath,
    "required_periods": REQ_PERIODS
})

# **Conjuntos de datos**

**Terminología**:
* Tipo 1: individuos tratados.
* Tipo 2: individuos de control (i.e. podrían haber sido tratados pero por alguna razón no lo fueron)
* Tipo 3: ni tratados ni de control.

##### **1. Separamos en tipo 1, tipo 2 y tipo 3.**

In [ ]:
type1_df, type2_df, type3_df = get_dfs(df, REQ_PERIODS)

# Chequeo para asegurarnos que los que tienen target 1 de tipo 2 se corresponden
# con el inicio_prog de la base original
assert(
    all(
        type2_df[type2_df['target'] == 1]['inicio_prog'] == 
        df[df['tipo'] == 2].groupby('id')['inicio_prog'].first()
    )
)

# Cantidad de inicios de programa. Esto nos dice cuántos duplicados hay de cada
# individuo de tipo 2y tipo 3.
min_inicio_prog = type1_df['inicio_prog'].min()
max_inicio_prog = type1_df['inicio_prog'].max()
amount_inicio_prog = max_inicio_prog - min_inicio_prog + 1

#### **2. Separamos en train (que es sobre el que despues se va a hacer KFold) y test.**

In [ ]:
# Cantidad de individuos de cada tipo (con los duplicados)
n1 = len(type1_df)
n2 = len(type2_df)
n3 = len(type3_df)

# Cantidad total de individuos en el dataset POST-TRANSFORMACION. Estas son las
# muestras que se van a usar en el proceso de entrenamiento y testing, por eso
# se tienen en cuenta los duplicados.
n = n1 + n2 + n3

# Tamaño de los conjuntos de entrenamiento y testeo
train_size = int(0.7 * n)
test_size = n - train_size

# En type2_df y type3_df, tenemos el mismo individuo pero con distinto inicio_prog
# y por lo tanto distintos valores de y(t-1), y(t-2), ..., y(t-required_periods)
# Lo que queremos es que todas las "copias" de un mismo individuo estén o todas
# en train o todas en test. Por lo tanto, la selección no va a ser sobre filas
# del dataframe sino sobre ids de individuos.
type3_ids = type3_df.index.unique()

# Todos los individuos de tipo 1 van al conjunto de entrenamiento, y los restantes
# son de tipo 3 elegidos al azar
# Dividimos por amount_inicio_prog porque vamos a seleccionar ids de individuos
# de tipo 3, no filas
# type3_train = int((train_size - n1) / amount_inicio_prog)
n_type3_train = 1000

# Ahora sí, seleccionamos aleatoriamente n3_train ids de individuos de tipo 3
# para el conjunto de entrenamiento
type3_train = np.random.choice(type3_ids, n_type3_train, replace=False)

# Y nos quedamos con las filas del dataframe de tipo 3 que tienen los ids seleccionados
type3_train_df = type3_df.loc[type3_train]

# Los ids que no están en type3_train son para el conjunto de testeo
n_type3_test = 500
type3_test = list(set(type3_ids) - set(type3_train))
type3_test = np.random.choice(type3_test, n_type3_test, replace=False)
type3_test_df = type3_df.loc[type3_test]

In [ ]:
train_df = pd.concat([type1_df, type3_train_df])
X_train_df, y_train_df = train_df[FEATS], train_df['target']
smote = SMOTE(sampling_strategy=0.6, k_neighbors=5, random_state=13)
X_train_df, y_train_df = smote.fit_resample(X_train_df, y_train_df)

test_df = pd.concat([type2_df, type3_test_df])
X_test_df, y_test_df = test_df[FEATS], test_df['target']

weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train_df), y=y_train_df
)

In [ ]:
y_train_df.value_counts() / len(y_train_df) * 100

In [ ]:
y_test_df.value_counts() / len(y_test_df) * 100

#### **3.Estandarizamos ambos conjuntos en base a los datos de entrenamiento.**

In [ ]:
data_preprocessor = DataPreprocessor(X_train_df, X_test_df, y_train_df, y_test_df)

X_train_df_scaled, X_test_df_scaled = data_preprocessor.scale_data()
X_train_df_scaled.head()

#### **4. Logueamos los datasets finales a MLFlow.**

In [ ]:
# We build the datasets with the target column to load them into mlflow
train_df = X_train_df_scaled.copy()
train_df['target'] = y_train_df
mlflow_logger.log_input(train_df, "train")

test_df = X_test_df_scaled.copy()
test_df['target'] = y_test_df
mlflow_logger.log_input(test_df, "test")

#### **5. Obtenemos estructura necesaria según la red que querramos usar y construimos Datasets**

In [ ]:
train_set, test_set = data_preprocessor.build_datasets(
    MODEL_ARCH, TEMP_FEATS, STAT_FEATS, scale=True
)

# **Búsqueda de hiperparámetros con Optuna**

#### **6. Dejamos seleccionado el constructor de modelo para la búsqueda de hiperparámetros**

In [ ]:
match MODEL_ARCH:
    case "lstm_v1":
        define_model = define_lstm_v1_model
    case "lstm_v2":
        define_model = define_lstm_v2_model
    case "gru":
        define_model = define_gru_model
    case "dense":
        define_model = define_dense_model
    case "fcn":
        define_model = define_fcn_model

#### **7. Definimos los parámetros de la búsqueda de hiperparámetros**

In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")
study_name = f"study_{timestamp}"
study_n_trials = 30

if len(METRICS) != len(DIRECTIONS):
    raise ValueError("The number of metrics and directions should be the same")

mlflow_logger.log_params({
    "optuna_study_name": study_name,
    "optuna_study_n_trials": study_n_trials,
    "objective_metrics": METRICS,
    "directions": DIRECTIONS
})

if "f_beta_score" in METRICS:
    mlflow_logger.log_param("f_beta_score", BETA)

In [ ]:
study = optuna.create_study(
    direction=DIRECTIONS[0] if len(METRICS) == 1 else None,
    directions=DIRECTIONS if len(METRICS) > 1 else None,
    storage=OPTUNA_STORAGE,
    study_name=study_name
)
study.set_metric_names(METRICS)

study.optimize(
    lambda trial: objective_cv(
        trial, define_model, train_set, weights, METRICS, beta=BETA
    ),
    n_trials=20,
    timeout=600,
    n_jobs=-1,
    callbacks=[MaxTrialsCallback(study_n_trials, states=(TrialState.COMPLETE,))],
    show_progress_bar=True
)

In [ ]:
if len(METRICS) > 1:
    print(f"Number of trials on the Pareto front: {len(study.best_trials)}")
    for i, (metric, direction) in enumerate(zip(METRICS, DIRECTIONS)):
        if direction == 'maximize':
            best_trial = max(study.best_trials, key=lambda t: t.values[i])
        elif direction == 'minimize':
            best_trial = min(study.best_trials, key=lambda t: t.values[i])
        print(f"Metric: {metric}")
        print(f"\tDirection: {direction}")
        print(f"\tTrial number: {best_trial.number}")
        print(f"\tValues: {best_trial.values}")
        print(f"\tParams: {best_trial.params}")
    
    fig = plot_pareto_front(study, target_names=METRICS)
    mlflow_logger.log_plot(fig, "pareto_front_plot.png")
    fig.show()

In [ ]:
best_trials_info = get_best_trials_info(study, METRICS)
best_trials_numbers = [trial['trial_number'] for trial in best_trials_info]
pprint.pp(f"Best trials info: {best_trials_info}")
pprint.pp(f"Best trials numbers: {best_trials_numbers}")

mlflow_logger.log_json(best_trials_info, "best_trials_info.json")
mlflow_logger.log_params({
    "best_trials_numbers": best_trials_numbers
})

In [ ]:
if len(best_trials_numbers) == 1:
    selected_trial = best_trials_numbers[0]
else:
    selected_trial = select_trial(best_trials_numbers)

mlflow_logger.log_param("selected_trial_number", selected_trial)

# **Entrenamiento del modelo con mejores hiperparámetros**

In [ ]:
params = study.trials[selected_trial].params

model = instantiate_model(MODEL_ARCH, params)
mlflow_logger.log_model_architecture(model)

# Common parameters for all models
epochs = params['n_epochs']

# optimizer_name and lr parameters are for specifying the optimizer
optimizer_name = params['optimizer']
lr = params['lr']
optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

# batch_size is for the training loop
batch_size = params['batch_size']
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True , num_workers=4)
test_loader  = DataLoader(test_set , batch_size=batch_size, shuffle=False, num_workers=4)

# loss function
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(weights[1], dtype=torch.float32))

In [ ]:
for epoch in tqdm(range(epochs)):
    print(f"Epoch {epoch} -----------------------------------------------------")
    avg_batch_loss_train = train_step(model, train_loader, loss_fn, optimizer)
    print(f"Average batch loss during training: {avg_batch_loss_train}")

    avg_batch_loss_test = validate_step(model, test_loader, loss_fn)
    print(f"Average batch loss during testing: {avg_batch_loss_test}")

In [ ]:
# for epoch in tqdm(range(epochs)):
#     print(f"Epoch {epoch} -----------------------------------------------------")
#     avg_batch_loss_train = train_step(model, train_loader, loss_fn, optimizer)
#     print(f"Average batch loss during training: {avg_batch_loss_train}")

#     avg_batch_loss_test = validate_step(model, test_loader, loss_fn)
#     print(f"Average batch loss during testing: {avg_batch_loss_test}")

#     metrics_values = validate_step_with_metrics(
#         model,
#         X_test_tensor,
#         y_test_tensor,
#         loss_fn,
#         METRICS,
#         beta=BETA,
#         train_features_mean=None
#     )
#     pprint.pp(metrics_values)
#     mlflow_logger.log_metrics(metrics_values, step=epoch)

# mlflow.pytorch.log_model(model, "trained_model")

# **Evaluación del modelo entrenado**

In [ ]:
X_train_tensor, y_train_tensor = train_set.tensors
X_test_tensor, y_test_tensor = test_set.tensors

In [ ]:
model.to('cpu')
y_test_pred = model(X_test_tensor.to('cpu'))
y_test_pred = predict(y_test_pred, loss_fn).squeeze()

In [ ]:
fig = confusion_matrix_plot(y_test_tensor, y_test_pred)
mlflow_logger.log_plot(fig, "confusion_matrix_plot.png")
fig.show()

In [ ]:
roc_curve_plot(y_test_tensor, y_test_pred)

In [ ]:
mlflow_logger.end_run()